In [4]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner
import random
import math, statistics
import numpy as np


Agent

In [27]:
class CaseAgent(Agent):
    """An agent with fixed initial wealth."""
    def __init__(self, unique_id, model, memory, actions_chosen = []):
        super().__init__(unique_id, model)
        self.wealth = 0
        self.memory = []
        self.prediction = 1
        self.result=0
        self.cases = []
        self.circumstances = []
        self.actions_chosen = []
        self.results_before = []
    def print(self): 
        print( "my__result", self.result, ".")
        
    def euclidean_distance(self,new_circumstance,old_circumstance):
        return math.sqrt(sum(pow(a-b,2) for a, b in zip(new_circumstance,old_circumstance)))

    def similarity(self,new_circumstance,old_circumstance):
        return 1/math.exp(self.euclidean_distance(self.new_circumstance,self.old_circumstance))
    
    def cases(dict):
        self.cases = {"result":"result", "action":"action", "circumstance":"circumstance"}
        return self.cases

    def CBU(self,action):
        for i in self.actions_chosen:
            self.results = self.result
            i.results_before.append(action.result)
        return sum(self.similarity(self.new_circumstance,self.old_circumstance)*(self.results_before-model.H))


    def make_prediction(self):
            # random choice for now
            
        prediction=random.randint(0,100)
        return prediction


    def evaluate_actions(self):                  #evaluate all the actions and choose the one which has highest CBU
        
        for action in self.actions:
            return self.CBU(action)
                
           
        else:
            return random.randint(0,100)
        
    def get_circumstances(self):
        return self.circumstances
    
    def get_result(self):
       
        result = max(1/2*(1-(model.realized_price - self.prediction)**2/7),0)   #calculate each agent's payoff
        return result
        


Memory

In [19]:
class memory(list):
    "agents have memories; memory is a list of all cases "
    def __init__(self,case = None):
        self.case = []
     #you can start a memory with contents if you want:
        if contents is not None:
            super().__init__(contents)
        else:
            super().__init__()
        
        
        self.default_value=default_value
        
        self.owner=owner
        
        self.max_length=max_length   
        
    def add_memory(self,time=0,result=None):
        if result is None:
            result = self.default_value
        
        c = GeneCase(time=time,result=result)
        dbp(2,"GeneMemory add case ", c.string())

        self.append(c)
        if len(self) > self.max_length:
            del self[0]
        
      

In [20]:
class case():
    def __init__(self,action=None, result=None, circumstance=None, time=0):
        self.action = action
        self.result = result
        self.circumstance = circumstance
        self.time = time
       
    def print(self):
        print("< i:",self.info," , t:", self.time, " , r:", self.result)
        
    def get_circumstance_set(self):
        self.circumstance = model.datacollector.get_model_vars_dataframe()["Price"]
        return self.circumstance
              
    def new_circumstance():
        return self.circumstance[-1]
    
    def old_circumstance():
        return self.circumstance[:-1]
    
    def get_action(self):
        self.action = model.datacollector.get_agent_vars_dataframe()["Action"]
        
    def get_result(self):
        self.result = model.datacollector.get_agent_vars_dataframe()["Result"]

Model

In [95]:
class CaseModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = 6
        # Create agents
        self.schedule = RandomActivation(self)
        self.predictions=[]
        self.results=[]
        self.wealths=[]
        self.realized_price = None
        self.agents=[] #ADP: have an agents vector directly in Model.
        self.tick=0    #ADP: keep track of time.
        self.H = 1000
          # Create agents
        for i in range(self.num_agents):
            a = CaseAgent(i, self, memory)
            self.schedule.add(a)
            self.agents.append(a)
        self.datacollector = DataCollector(
            model_reporters={"Price": "realized_price"},
            agent_reporters={"Wealth": "wealth", "Result": "result", "Action": "prediction", "Cases":"cases"})
        
    def reset(self):
        self.predictions = []
        self.results = []
    def calculate_price(self):
        realized_price = round(20/21*(statistics.mean(self.predictions)+3))
        return realized_price
    def update_H(self):
        self.H = (self.H)**0.95
        return self.H
    def step(self):
        '''Advance the model by one step.'''
        self.schedule.step()
        self.tick += 1 # ADP: iterate time.
        previous_price = self.realized_price
        for i in self.agents:
            model.H = model.update_H()
            i.new_circumstance = previous_price  #observe new circumstance
            i.old_circumstance = [0, 0] #recall old circumstances
            
            
            
            i.prediction = i.make_prediction()        #agents make predictions for each round "action"
            model.predictions.append(i.prediction)
            
        self.realized_price = self.calculate_price()    #model calculate the market price  "circumstance" 
        
        for i in self.agents: 
            i.result = i.get_result()                 #agents get their payoff for each round  "result"
            model.results.append(i.result)
            
            i.new_case = {"action": i.prediction,
                        "result": i.result,
                        "circumstance": previous_price}
            i.cases.append(i.new_case)
            print(i.cases[:3]) 
      
           
            #i.actions_chosen.append(i.prediction)
            i.wealth = i.wealth + i.result             #cumulative wealth
           # i.memory = i.memory.add_memory()
        self.datacollector.collect(self)    
       
        
       
        print( "Time:", self.tick, ".") 
        print( "Model_predictions:", self.predictions, ".")  
        print( "Model_results:", self.results, ".")
      
       
        self.reset() 
    def print(self): 
        # ADP: Give every object a "print" function.
     
        print( "my__prediction", self.predictions, ".")
        print( "realized_price", self.realized_price, ".")
       
   

Run the model

In [96]:
model = CaseModel(50)
for i in range(50):
    
    model.step()
    model.print()

[{'action': 81, 'result': 0, 'circumstance': None}]
[{'action': 49, 'result': 0.4285714285714286, 'circumstance': None}]
[{'action': 65, 'result': 0, 'circumstance': None}]
[{'action': 33, 'result': 0, 'circumstance': None}]
[{'action': 52, 'result': 0, 'circumstance': None}]
[{'action': 6, 'result': 0, 'circumstance': None}]
Time: 1 .
Model_predictions: [81, 49, 65, 33, 52, 6] .
Model_results: [0, 0.4285714285714286, 0, 0, 0, 0] .
my__prediction [] .
realized_price 48 .
[{'action': 81, 'result': 0, 'circumstance': None}, {'action': 72, 'result': 0, 'circumstance': 48}]
[{'action': 49, 'result': 0.4285714285714286, 'circumstance': None}, {'action': 9, 'result': 0, 'circumstance': 48}]
[{'action': 65, 'result': 0, 'circumstance': None}, {'action': 85, 'result': 0, 'circumstance': 48}]
[{'action': 33, 'result': 0, 'circumstance': None}, {'action': 28, 'result': 0, 'circumstance': 48}]
[{'action': 52, 'result': 0, 'circumstance': None}, {'action': 56, 'result': 0.5, 'circumstance': 48}]
[

In [43]:
agent1 = CaseAgent(1,CaseModel,memory)
model.step()
a=agent1.cases
print(a)

Time: 59 .
Model_predictions: [56, 93, 20, 40, 100, 82] .
Model_results: [0, 0, 0, 0, 0, 0] .
[]


In [58]:

print(agent(unique_id=0).cases)

NameError: name 'agent' is not defined